In [1]:
import json,openai,os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation(input):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": input}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "指定した場所の現在の天気を取得",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "都市と州",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    message = response["choices"][0]["message"]
    print("message>>>\n", message, "\n\n")

    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": input},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print("response>>>\n", run_conversation("ボストンの天気はどうですか？")["choices"][0]["message"]["content"], "\n\n")

message>>>
 {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"Boston\"\n}"
  }
} 


response>>>
 ボストンの現在の天気は晴れで、風の強い日です。気温は72です。 




In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, Tool

In [5]:
# 同じ関数
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

tools = [
    Tool(
        name='get_current_weather',
        func=get_current_weather,
        description='Get the current weather in a given location'
    )
]

chat = ChatOpenAI(temperature=0.)
agent = initialize_agent(
    tools,chat,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
# result = agent.run("ボストンの天気はどうですか？")
result = agent.run("天気はどうですか？")
print(result)



> Entering new  chain...

Invoking: `get_current_weather` with `{'location': '東京'}`


{"location": "\u6771\u4eac", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}東京の天気は晴れで、風が強いです。現在の気温は72°Fです。

> Finished chain.
東京の天気は晴れで、風が強いです。現在の気温は72°Fです。
